$$ A(M) = x^{rk(M)}\cdot \hat{Q}_M(x^{-2})$$

$$A(M\setminus e) = (x + x^{-1})\cdot A(M/e) + A(M) + \sum_{G \ni e;\ 
e\notin coloops(G)
}{A(M/G)\cdot\tau(M^G_e)}$$

In [73]:

R = PolynomialRing(ZZ, 't')
t = R.gen()

def cmp_elements_key(x):
    return (isinstance(x, str), x)

def characteristic_polynomial(M, la=None):
    R = ZZ['l']
    w = whitney_numbers(M)
    w.reverse()
    chi = R(w)
    if la is not None:
        return chi(la)
    return chi

def whitney_numbers(M):
    abs_w = [0] * (M.rank() + 1)
    for S in no_broken_circuits_sets_iterator(M):
        abs_w[len(S)] += 1
    return [ZZ((-1)**i * val) for i, val in enumerate(abs_w) if val != 0]

def no_broken_circuits_sets_iterator(M, ordering=None):
    if M.loops():
        return
    if ordering is None:
        rev_order = sorted(M.groundset(), key=cmp_elements_key, reverse=True)
    else:
        if frozenset(ordering) != M.groundset():
            raise ValueError("not an ordering of the groundset")
        rev_order = list(reversed(ordering))
    
    Tmax = len(rev_order)
    reverse_dict = {value: key for key, value in enumerate(rev_order)}
    yield frozenset()
    next_level = [[val] for val in rev_order]
    level = -1
    
    while next_level:
        cur_level = next_level
        next_level = []
        level += 1
        for H in cur_level:
            tp = reverse_dict[H[level]] + 1
            is_indep = True
            Ht = [None] * (Tmax - tp)
            for i in range(tp, Tmax):
                temp = H + [rev_order[i]]
                if not M._is_independent(frozenset(temp)):
                    is_indep = False
                    break
                Ht[i - tp] = temp
            if is_indep:
                yield frozenset(H)
                next_level.extend(Ht)

def is_paving(M):
    n = M.size()
    r = M.rank()
    return (len(M.independent_r_sets(r-1)) == binomial(n, r-1))

def q_kl(k, h):
    return kazhdan_lusztig_inverse_uniform(k, h+1) - kazhdan_lusztig_inverse_uniform(k-1, h)

def kl_inverse_fast(M):
    if M.loops(): return R(0)
    k, n = M.rank(), M.size()
    if k == n or k == 0: return R(1)
    if not M.is_connected():
        ans = R(1)
        CC = M.components()
        for N in CC:
            res = M.delete(M.groundset() - N)
            ans = ans * kl_inverse_fast(res)
        return ans

    if is_paving(M):
        return kl_inverse_paving(M)
    if is_paving(M.dual()):
        return kl_inverse_copaving(M)
        
    LF = M.lattice_of_flats()
    ans = R(0)
    for F in LF:
        if len(F) != n:
            Res = M.delete(M.groundset() - F)
            Con = M.contract(F)
            chi = characteristic_polynomial(Con)(1/t) * t**(Con.rank())
            PPP = kl_inverse_fast(Res)(t) * (-1)**(Res.rank())
            ans = ans + chi * PPP
    assert (t**k * ans(1/t)).numerator() == -ans(t)
    ans = ans.numerator() * (-1)**(k+1)
    return ans.truncate((k+1)//2)

def kazhdan_lusztig_inverse_uniform(k, n):
    if k == n:
        return R(1)
    d = k
    m = n - d
    ans = 0
    for j in range((d-1)//2 + 1):
        ans = ans + m * (d-2*j)/((m+j) * (m+d-j)) * binomial(d, j) * t**j
    return ans * binomial(m+d, d)

def kl_inverse_paving(M):
    assert is_paving(M)
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.hyperplanes():
        h = len(H)
        if h >= k:
            ans = ans - q_kl(k, h)
    return ans

def kl_inverse_copaving(M):
    assert is_paving(M.dual())
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.dual().hyperplanes():
        h = len(H)
        if h >= n-k:
            ans = ans - kli_vtilde_dual(n-k, h, n) + kazhdan_lusztig_inverse_uniform(h-n+k+1, h) * kazhdan_lusztig_inverse_uniform(n-h-1, n-h)
    return ans

def kli_vtilde_dual(k, h, n):
    return helper1(n-k, h, n)

def helper1(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_inverse_uniform(k, n)
    ans2 = helper2(c, k, n)
    ans3 = kazhdan_lusztig_inverse_uniform(k-c+1, h) * kazhdan_lusztig_inverse_uniform(c-1, c)
    return ans1 - ans2 + ans3

def helper2(c, k, n):
    h = n - c
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * (-1)**(c-1+j) * kazhdan_lusztig_inverse_uniform(c-1, c) * t**(k-c-j+1) * chuly(k-c-j+1, n-c-j)(1/t)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(i+j) * t**(k-i-j) * helper4(c, k, n, i, j)(1/t)
    ans = ans.numerator().truncate((k-1)//2 + 1)
    if ans[0] < 0:
        ans = -ans
    return ans

def helper3(c, k, n):
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * kazhdan_lusztig_uniform_matroid(c-1, c) * (-1)**(k-c-j+1) * kazhdan_lusztig_inverse_uniform(k-c-j+1, n-c-j)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(k-i-j) * helper2(c-i, k-i-j, n-i-j)
    return -ans

def helper4(c, k, n, i, j):
    ans = 0
    for l in range(c-i-1):
        ans = ans + (-1)**l * (t-1)**(max(n-i-j-l-1, 0))
    for u in range(n-k-1):
        ans = doit_once(ans)
    return ans

def chuly(a, b):
    ans = (t-1)**b
    for i in range(b-a):
        ans = doit_once(ans)
    return ans

def doit_once(p):
    p = p // t**2
    p = p * t
    p = p - p(1)
    return p

def lorenzo(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_uniform_matroid(k, n) + kazhdan_lusztig_uniform_matroid(k-c+1, h) * kazhdan_lusztig_uniform_matroid(c-1, c)
    ans2 = helper3(c, k, n)
    return ans1 - ans2

In [74]:
P = LaurentPolynomialRing(ZZ, 'x')

def kl(M):
    return M.lattice_of_flats().kazhdan_lusztig_polynomial()

invkl = kl_inverse_fast

def tau(matroid):
    r = matroid.rank()
    if r % 2 == 0:
        return 0
    klm = R(kl(matroid))
    if len(klm.list()) <= r//2:
        return 0
    return klm.list()[r//2]

def qhat(M):
    return (-1) ^ M.rank() * kl_inverse_fast(M)

def the_set_S(flats, e):
    return set(F for F in flats if e in F and frozenset(set(F) - {e}) not in flats)

In [75]:
def A(L):
    return (x^L.rank() * qhat(L)(x^-2)).expand()

def delform_rhs(M, e):
    r = M.rank()
    x = P.gen()
    rhs = (x + x^-1) * A(M.contract(e)) + A(M)
    print(M, rhs.expand())
    for G in the_set_S(M.lattice_of_flats(), e):
        Mminor = M.minor(deletions = M.groundset() - G, contractions = e)
        tauu = tau(Mminor)
        Am = A(M.contract(G))
        if 1:
            print(f"Flat: {G}, A(M.contract(G)): {Am}, "
                f"KL(): {kl(Mminor)}, Tau: {tauu}")
            rhs = rhs + Am * tauu
    return rhs

In [ ]:
M = matroids.CompleteGraphic(6)
e = next(x for x in M.groundset() if x in M.coloops())
L = M.lattice_of_flats()

In [77]:
[A(M.delete(e)), A(M), A(M.contract(e)), ((x + x^-1) * A(M.contract(e))).expand()]

[-96*x^5 - 76*x^3 - 13*x,
 -120*x^5 - 86*x^3 - 15*x,
 24*x^4 + 10*x^2,
 24*x^5 + 34*x^3 + 10*x]

In [78]:
rhs = delform_rhs(M, e)
rhs.expand()

M(K6): Graphic matroid of rank 5 on 15 elements -96*x^5 - 52*x^3 - 5*x
Flat: frozenset({0, 11, 3, 7}), A(M.contract(G)): 2*x^2, KL(): 1, Tau: 0
Flat: frozenset({0, 1, 2, 5, 6, 9, 14}), A(M.contract(G)): -x, KL(): 1, Tau: 0
Flat: frozenset({0, 3, 4, 7, 8, 9, 14}), A(M.contract(G)): -x, KL(): 1, Tau: 0
Flat: frozenset({0, 2, 4, 6, 8, 13}), A(M.contract(G)): 2*x^2, KL(): 1, Tau: 0
Flat: frozenset({0, 2, 3, 6, 7, 12}), A(M.contract(G)): 2*x^2, KL(): 1, Tau: 0
Flat: frozenset({0, 1, 5, 14}), A(M.contract(G)): 2*x^2, KL(): 1, Tau: 0
Flat: frozenset({0, 1, 3, 5, 7, 10}), A(M.contract(G)): 2*x^2, KL(): 1, Tau: 0
Flat: frozenset({0, 2, 4, 6, 8, 10, 13}), A(M.contract(G)): -x, KL(): 1, Tau: 0
Flat: frozenset({0, 1, 2, 5, 6, 9}), A(M.contract(G)): 2*x^2, KL(): 1, Tau: 0
Flat: frozenset({0, 1, 2, 3, 5, 6, 7, 9, 10, 12}), A(M.contract(G)): -x, KL(): q + 1, Tau: 1
Flat: frozenset({0, 2, 11, 6}), A(M.contract(G)): 2*x^2, KL(): 1, Tau: 0
Flat: frozenset({0, 2, 6}), A(M.contract(G)): -6*x^3 - x, KL(): 

-96*x^5 - 76*x^3 - 13*x

$$Q_{M\setminus e}(x) = - (1+ x)Q_{M/e}(x) + Q_M (x) + \sum_{G\ni e; e\notin coloops(M^G)}{(-1)^g x^{g/2}
Q_{M/G}(x)\cdot\tau(M^G_e)}$$